<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第二章：文本处理
本notebook需要使用的库和对应版本

In [1]:
from importlib.metadata import version

print("torch version: ", version('torch'))
print("tiktoken version: ", version('tiktoken'))


torch version:  2.2.1+cu118
tiktoken version:  0.6.0


- 本章介绍数据准备和采样，为大语言模型(LLM)的输入数据做好"准备"

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">
</p>

## 2.1 理解词嵌入
- 本节没有代码
- 存在各种形式的嵌入，如视频嵌入、音频嵌入等，本书聚焦在文本嵌入。对于嵌入/embeddings可以理解为一种可表示原始数据对象的向量数据，因为训练过程就是一个计算的过程，需要将各类训练数据转换为可计算的形式。

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">
</p>

- LLMs在高维空间中与嵌入交互，如上千个维度
- 人类只能观测到1、2、3维空间，所以不能将高维空间可视化，但下图中二维空间可以大致表明嵌入空间的概念

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">
</p>

## 2.2 文本分词
- 本节对文本进行分词，或者理解为标记化处理，即将一段文本序列分解为更小的单元，如单个词或标点符号。下图中就是将一个句子按单词进行分词，分词后每个单词对应一个id，可以理解为该单词在整个vocab中的索引，也可以理解为分类任务中的一个类别标签，因为时数值，可以通过embedding层转换为嵌入向量

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">
</p>

- 加载准备处理的数据
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict)是一个公开、无版权的短篇小说

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)  # 下载文件到指定地点

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters: ", len(raw_text))
print(raw_text[:99])

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对文本进行分词和嵌入化，以其可作为LLMs的输入
- 基于一些简单的示例文本开发一个简单的分词器，之后可以应用到上面的文本中
- 以下正则表达式将按空白字符进行分割

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 不仅想按空白字符分割，还想按逗号和句号分割，所以让修改正则表达式以实现这一点

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 上述实现会产生控制符，将其删除

In [ ]:
# 消除每个元素前后的空字符，然后将所有为空字符的元素过滤掉
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 上述实现不错，但是还要处理如句号、问号等标点符号

In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 上述实现的分词方式已经可以应用到原始文本了

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">
</p>

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 查看所有tokens的数量

In [9]:
print(len(preprocessed))

4690


## 2.3 将tokens隐射为token IDs
- tokens转换为token IDs便于通过嵌入层进行计算

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">
</p>

- 基于从raw text中提取的tokens可以创建一个有所有唯一的token组成的词语表/vocab

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


- 构建一个key为token，value为id数值的字典，即一个便于将token隐射到id的映射关系

In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下是词汇表中的前五十个实体

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下图阐述了使用一个小的词汇表对一段短文进行分词的过程
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">
</p>


- 将上述关于分词的所有内容封装为一个类

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 替换指定标点符号前的空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- encode方法将文本转换为token IDs
- decode方法将token IDs转换会文本
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">
</p>

- 可使用分词器将文本编码为数值序列
- 数字序列进行嵌入化后作为LLM的输入

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """
           "It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride.
       """
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 将数值序列转换为文本

In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文tokens
- 在文本中可能会存在一些不知道的词汇，也需要一些特殊词汇表示文本结束等，就需要设置一些特殊tokens；下图中的|unk|、|endoftext|就是分别表示不知道和文本结束的特俗tokens
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">
</p>

- 一些分词器使用特殊tokens帮助LLM理解额外的上下文，具体的特殊tokens有：
  - [BOS]：开始标记，表示文本的开始
  - [EOS]：结束标记，表示文本的结束(通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书籍等）
  - [PAD]：LLMs训练的一个batch中，多条样本序列长度一般是不同的，一般会将所有样本长度都pad到最长的样本长度，就是使用[PAD]来填充
  - [UNK]：当文本中存在词汇表中不存在的token时，就用此未知token表示
- 注意：GPT-2不使用上述的任何特征字符，为了降低复杂度仅使用<|endoftext|>
  - <|endoftext|>相当于上述提到的[EOS]
  - GPT也是使用<|endoftext|>作为填充字符，因为训练过程计算注意力或损失时会使用mask排除padding的tokens，所以padding的token具体是什么其实不重要
- GPT-2 不使用 <UNK> 表示词汇表外的单词；相反，GPT-2 使用字节对编码（BPE）分词器，它将单词分解为子词单元，我们将在后面的章节中讨论这一点。
- 在两个独立、无关的文本之间使用<|endoftext|>进行区分
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">
</p>

- 看看分词以下文本会发生什么

In [17]:
tokenizer = SimpleTokenizerV1(vocab)
text = "Hello, do you like tea. Is this-- a test?"
tokenizer.encode(text)



KeyError: 'Hello'

- 我们看到产生了一个错误，因为词汇表中没有“Hello”这个token
- 为了处理这样的问题，可以将<UNK>添加到词汇表中，当遇到词汇表中没有的token时，就用<UNK>表示
- 除了<UNK>外，还可以将<|endoftext|>一并添加，它在 GPT-2 训练中用来表示文本的结束（它也用于连接文本之间，比如当我们的训练数据集包含多篇文章、书籍等时）。

In [30]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<unk>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

len(vocab.items())

1132

In [31]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<unk>', 1131)


- 我们需要相应调整tokenizer，使其知道什么时候、应该如何使用<UNK>

In [35]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<unk>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- 使用调整后的分词器对文本进行分词

In [36]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = text1 + " <|endoftext|> " + text2

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [37]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [38]:
tokenizer.decode(tokenizer.encode(text))

'<unk>, do you like tea? <|endoftext|> In the sunlit terraces of the <unk>.'

## 2.5 字节对编码(BytePair Encoding, BPE)
- GPT-2使用BPE作为分词器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元甚至单个字符，使其能够处理词汇表外的单词
- 例如，如果GPT-2的词汇表没有"unfamiliarword"这个词，它可能会将其标记为["unfam", "iliar", "word"]或其他子词分解，这取决于它训练的BPE合并
- 原始的BPE分词器可以在这里找到：https://github.com/openai/gpt-2/blob/master/src/encoder.py
- 本章使用OpenAI开源的tiktoken库中的BPE分词器，它在Rust中实现其核心算法以提高计算性能
- 在[./bytepair_encoder](../02_bonus_bytepair-encoder)中创建了一个notebook，并排比较了这两种实现（在样本文本上，tiktoken大约快5倍）


In [ ]:
# pip install tiktoken

In [42]:
import importlib
import tiktoken

print("tiktoken version: ", importlib.metadata.version("tiktoken"))

tiktoken version:  0.6.0


In [43]:
tokenizer = tiktoken.get_encoding("gpt2")

In [44]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [45]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE分词器将不知道的单词分解为子词或字符
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">
</p>


In [54]:
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a significant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~") + 1)) + list(range(ord("¡"), ord("¬") + 1)) + list(range(ord("®"), ord("ÿ") + 1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8 + n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

In [56]:
bytes_to_unicode()[33]

'!'

In [55]:
for key, item in bytes_to_unicode().items():
    print(key, item)


33 !
34 "
35 #
36 $
37 %
38 &
39 '
40 (
41 )
42 *
43 +
44 ,
45 -
46 .
47 /
48 0
49 1
50 2
51 3
52 4
53 5
54 6
55 7
56 8
57 9
58 :
59 ;
60 <
61 =
62 >
63 ?
64 @
65 A
66 B
67 C
68 D
69 E
70 F
71 G
72 H
73 I
74 J
75 K
76 L
77 M
78 N
79 O
80 P
81 Q
82 R
83 S
84 T
85 U
86 V
87 W
88 X
89 Y
90 Z
91 [
92 \
93 ]
94 ^
95 _
96 `
97 a
98 b
99 c
100 d
101 e
102 f
103 g
104 h
105 i
106 j
107 k
108 l
109 m
110 n
111 o
112 p
113 q
114 r
115 s
116 t
117 u
118 v
119 w
120 x
121 y
122 z
123 {
124 |
125 }
126 ~
161 ¡
162 ¢
163 £
164 ¤
165 ¥
166 ¦
167 §
168 ¨
169 ©
170 ª
171 «
172 ¬
174 ®
175 ¯
176 °
177 ±
178 ²
179 ³
180 ´
181 µ
182 ¶
183 ·
184 ¸
185 ¹
186 º
187 »
188 ¼
189 ½
190 ¾
191 ¿
192 À
193 Á
194 Â
195 Ã
196 Ä
197 Å
198 Æ
199 Ç
200 È
201 É
202 Ê
203 Ë
204 Ì
205 Í
206 Î
207 Ï
208 Ð
209 Ñ
210 Ò
211 Ó
212 Ô
213 Õ
214 Ö
215 ×
216 Ø
217 Ù
218 Ú
219 Û
220 Ü
221 Ý
222 Þ
223 ß
224 à
225 á
226 â
227 ã
228 ä
229 å
230 æ
231 ç
232 è
233 é
234 ê
235 ë
236 ì
237 í
238 î
239 ï
240 ð
241 ñ
242 ò
243 ó
244 ô
245 õ